# Sequential removal of links and resiliency testing

- Disruption of a network by removal of links, based on:
    + Sum of betweeness centrality of from and to nodes
    + Link length
    + Volume of commodity flow
- Calculation of performance in terms of cost and unmet demand by re-running disrupted network 
- Plot link removal along x-axis and performance on y-axis, comparing networks of differing evenness. Dynamic report generated in an RMarkdown automatically from this Notebook.

**Assumptions**

- Working in a Python 3.x environment for this notebook
    + Refer to the README in this repository for instructions on setup of all dependencies with `conda`
- Access to ArcGIS license server if necessary 

*Reference*

- [NetworkX Documentation](https://networkx.github.io/documentation/stable/tutorial.html)

In [1]:
import pandas as pd
import geopandas as gpd
import sqlalchemy 
import networkx as nx
import os
import pickle
import momepy # for conversion from geopandas GeoDataFrame to networkX Graph

import resiliency_disruptions

# Uses Reference Scenario 1 as an example. Modify `scen_name` and `scen_path` for your scenario.
scen_name = 'ga_freight_new_port - Copy'

scen_path = os.path.join("C:\\FTOT\\scenarios\\FHWA\\generic_freight", scen_name)

shp_path = os.path.join(scen_path, 'temp_networkx_shp_files')

picklename = os.path.join(scen_path, 'BetweenessG.pickle')

if not os.path.exists(shp_path):
    print('Please modify the FTOT code using the `ftot_networkx.py` and `ftot_routing.py` scripts in this repository and run the scenario again.')

In [2]:
# Read in prepared betweeness centrality and road network graph data. 
# If these don't exist, the following steps will create them
if os.path.exists(picklename):
    file = open(picklename, 'rb')
    betweenness_dict_road = pickle.load(file)
    G_road = pickle.load(file)

In [3]:
# Start by using betweeness centrality calculation using networkX
if not os.path.exists(picklename):
    road = gpd.read_file(os.path.join(shp_path, 'road.shp'))
    
    # convert from geodataframe to Graph for networkX
    G_road = momepy.gdf_to_nx(road, approach='primal')
    
    # Process the networkX graph
    G_road = nx.convert_node_labels_to_integers(G_road, first_label=0, ordering='default', label_attribute="xy_coord_label")


In [4]:
# Run betweenness centrality on the NetworkX graph
# Note: This step might take 20+ minutes
# Run if pickle not available
if not os.path.exists(picklename):
    print('Running Betweeness Centrality calculations. This might take more than 20 minutes.')
    betweenness_dict_road = nx.betweenness_centrality(G_road, normalized=False, weight='MILES')
    print('Completed Betweeness Centrality calculations.')


In [5]:
# Save with pickle
# On load, need to know that there are two objects in this pickle, the betweeness centrality dict and the network G
if not os.path.exists(picklename):
    with open(picklename, 'wb') as handle:
        pickle.dump(betweenness_dict_road, handle)
        pickle.dump(G_road, handle)

## Join Betweeness Centrality calculations to edges 

- Sum BC for each node of a link
- Create data frame for repeated link removal

In [15]:
# Read in FTOT data
print(scen_path)
db_name = 'main.db'

db_path = 'sqlite:///' + os.path.join(scen_path, db_name)

engine = sqlalchemy.create_engine(db_path)

table_name = 'networkx_edges'
nx_edges = pd.read_sql_table(table_name, engine)

table_name = 'networkx_nodes'
nx_nodes = pd.read_sql_table(table_name, engine)

table_name = 'optimal_variables'
optimal_vars = pd.read_sql_table(table_name, engine)

C:\FTOT\scenarios\FHWA\generic_freight\ga_freight_new_port - Copy


In [16]:
optimal_vars

,variable_type,var_id,variable_value,converted_capacity,converted_volume,converted_capac_minus_volume,edge_type,commodity_name,o_facility,d_facility,...,units,variable_name,nx_edge_id,mode,mode_oid,length,original_facility,final_facility,prior_edge,distance_travelled
0,Edge,1,924359.0,None,None,None,connector,freight_bulk,rmp_13051,rmp_13051,...,metric_ton,Edge_1,None,None,None,NaN,None,None,None,None
1,Edge,2,63317.0,None,None,None,connector,freight_parcel,dest_01073,dest_01073,...,metric_ton,Edge_2,None,None,None,NaN,None,None,None,None
2,Edge,3,42425.0,None,None,None,connector,freight_parcel,dest_01089,dest_01089,...,metric_ton,Edge_3,None,None,None,NaN,None,None,None,None
3,Edge,4,6107.0,None,None,None,connector,freight_parcel,dest_21111,dest_21111,...,metric_ton,Edge_4,None,None,None,NaN,None,None,None,None
4,Edge,5,10240.0,None,None,None,connector,freight_parcel,dest_13059,dest_13059,...,metric_ton,Edge_5,None,None,None,NaN,None,None,None,None
5,Edge,6,5785.0,None,None,None,connector,freight_parcel,dest_39061,dest_39061,...,metric_ton,Edge_6,None,None,None,NaN,None,None,None,None
6,Edge,7,84895.0,None,None,None,connector,freight_parcel,dest_13121,dest_13121,...,metric_ton,Edge_7,None,None,None,NaN,None,None,None,None
7,Edge,8,107028.0,None,None,None,connector,freight_parcel,dest_37021,dest_37021,...,metric_ton,Edge_8,None,None,None,NaN,None,None,None,None
8,Edge,9,26355.0,None,None,None,connector,freight_parcel,dest_37081,dest_37081,...,metric_ton,Edge_9,None,None,None,NaN,None,None,None,None
9,Edge,10,56889.0,None,None,None,connector,freight_parcel,dest_37119,dest_37119,...,metric_ton,Edge_10,None,None,None,NaN,None,None,None,None


In [7]:
road_orig_label = gpd.read_file(os.path.join(shp_path, 'road.shp'))
# convert from geodataframe to Graph for networkX
G_road_orig_label = momepy.gdf_to_nx(road_orig_label, approach='primal')

In [8]:
road_orig_label_nodes = list(G_road_orig_label.nodes) # these values are the shape_x and shape_y values in `networkx_nodes`. 
# Use that to get node_id from networkx_edges in the database,
# Then use those id values to get edges info
# Then line up the new integer labels with this list of ids to get betweeness centrality for each node

In [9]:
# Make the betweeness_centrality values as the framework to join in shape_x, shape_y, and node_id
bc_df_road = pd.DataFrame.from_dict(betweenness_dict_road, orient = 'index')
bc_df_road = bc_df_road.rename(columns = {0: 'BC'})

In [10]:
node_shape_df_road = pd.DataFrame(road_orig_label_nodes)

bc_shape_df_road = pd.concat([bc_df_road, node_shape_df_road], axis = 1)
bc_shape_df_road = bc_shape_df_road.rename(columns = {0: 'shape_x', 1: 'shape_y'})

# Now add node_id from networkx_nodes, using pandas merge with left join.
# Use both shape_x and shape_y to identify the nodes correctly
# Union of both prod and crude now

bc_node_df = pd.merge(bc_shape_df_road, nx_nodes, on = ['shape_x', 'shape_y'], how = 'left')

In [12]:
# Now use this data frame to populate a data frame of edges. 
# We will want the following from networkx_edges:
# edge_id, from_node_id, to_node_id, mode_source, miles, mode_source_oid, 
# Then using the node_id column in the new bc_node_df, add these:
# from_node_BC, to_node_BC
# and sum those for sum_node_BC
merge_from = pd.merge(nx_edges, bc_node_df[['BC','node_id']],
                      left_on = 'from_node_id',
                      right_on = 'node_id',
                      how = 'left')
merge_from = merge_from.rename(columns = {'BC': 'from_node_BC'})

merge_to = pd.merge(merge_from, bc_node_df[['BC','node_id']],
                    left_on = 'to_node_id',
                    right_on = 'node_id',
                    how = 'left')
merge_to = merge_to.rename(columns = {'BC': 'to_node_BC'})

# Sum the BC values

merge_to['sum_BC'] = merge_to.filter(like = "node_BC").sum(axis = 1)

# Then from optimal_variables, get variable_name, nc_edge_id, mode, mode_oid, miles,
# variable_value, converted_capacity, and converted_volume

use_opt_vars = ['variable_type',
               'var_id',
               'variable_value',
                'variable_name',
                'nx_edge_id',
                'mode_oid',
                'converted_capacity',
                'converted_volume'
               ]

merge_opt = pd.merge(merge_to, optimal_vars[use_opt_vars],
                    left_on = 'edge_id',
                    right_on = 'nx_edge_id',
                    how = 'left')

ValueError: You are trying to merge on int64 and object columns for key 'edge_id'. If you wish to proceed you should use pd.concat

In [13]:
merge_to

,edge_id,from_node_id,to_node_id,artificial,mode_source,mode_source_oid,length,route_cost_scaling,capacity,volume,VCR,from_node_BC,node_id_x,to_node_BC,node_id_y,sum_BC
0,1,0,2199,2,rail,178,0.013221,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
1,2,0,53852,2,water,1373,0.351856,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,3,1,2200,2,rail,179,0.064799,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
3,4,2,2201,2,rail,180,0.292158,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
4,5,3,2202,2,rail,181,0.394701,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120553,120554,57324,1734,1,water,6192,1.822503,1.3,NaN,NaN,NaN,NaN,NaN,0.0,1734.0,0.0
120554,120555,57324,1823,0,water,6181,4.560031,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
120555,120556,57325,55441,0,water,6184,41.134074,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
120556,120557,57325,1736,1,water,6194,1.573979,1.3,NaN,NaN,NaN,NaN,NaN,0.0,1736.0,0.0


In [14]:
optimal_vars[use_opt_vars]

,variable_type,var_id,variable_value,variable_name,nx_edge_id,mode_oid,converted_capacity,converted_volume
0,Edge,1,924359.0,Edge_1,None,None,None,None
1,Edge,2,63317.0,Edge_2,None,None,None,None
2,Edge,3,42425.0,Edge_3,None,None,None,None
3,Edge,4,6107.0,Edge_4,None,None,None,None
4,Edge,5,10240.0,Edge_5,None,None,None,None
5,Edge,6,5785.0,Edge_6,None,None,None,None
6,Edge,7,84895.0,Edge_7,None,None,None,None
7,Edge,8,107028.0,Edge_8,None,None,None,None
8,Edge,9,26355.0,Edge_9,None,None,None,None
9,Edge,10,56889.0,Edge_10,None,None,None,None


In [ ]:
merge_opt.head(3)

In [ ]:
# Create ranked lists of edges to remove.
# First, keep only edges in the optimal solution.
# Then rank by sum_BC. Then just keep the columns we need, and reset the index.
use_cols = ['edge_id', 'from_node_id', 'to_node_id', 'miles', 'capacity', 'volume', 'sum_BC',
           'variable_type', 'variable_value', 'variable_name', 'nx_edge_id', 'mode_oid', 'converted_capacity', 'converted_volume']

edges_remove = merge_opt[merge_opt['variable_value'] > 0].sort_values(by = 'sum_BC', ascending = False).filter(items = use_cols).reset_index()

edges_remove.to_csv(os.path.join(scen_path, 'Edges_to_Remove.csv'),
                   index = False)

edges_remove.head(3)

## Create Scenarios, Disrupt, Run FTOT

Create disrupted network by copying everyhing in `scen_path` to a new directory

Then overwrites the `networkx_edges` tables in that main.db, with the disrupted versions.

##### Assuptions:

  1. ArcGIS with 64-bit geoprocessing is installed
  2. The FTOT version being used has been modified according to the `README` in this directory


In [ ]:
disrupt_type = 'BC' # Can disrupt basaed on betweeness centrality or volume, 'V'
disrupt_steps = 25  # This is the number of steps to use. Recommend at least 25.

resiliency_disruptions.make_disruption_scenarios(disrupt_type, disrupt_steps, scen_path)

In [ ]:
resiliency_disruptions.disrupt_network(disrupt_type, disrupt_steps, scen_path, edges_remove)

In [ ]:
PYTHON = r"C:\FTOT\python3_env\python.exe"
repo_location = %pwd
repo_location = os.path.split(repo_location)[0] 
FTOT = r"C:\FTOT\program\ftot.py" # Optionally: os.path.join(repo_location, 'program', 'ftot.py')
print(FTOT)

In [ ]:
# Begin running O steps of FTOT on the disupted scenarios
# This may take several hours, depending on size of the network and number of steps

results = resiliency_disruptions.run_o_steps(disrupt_type, disrupt_steps, scen_path, PYTHON, FTOT)

In [ ]:
results

#### Optional: Repeat with volume-based disruptions

Creates a separate directory tree for the volume-based disruptions, and carries out the disruption steps on that set.

Set the variable `DO_VOLUME` to `True` to run the following steps

In [ ]:
DO_VOLUME = False

if DO_VOLUME:

    disrupt_type = 'V'
    disrupt_steps = 50

    resiliency_disruptions.make_disruption_scenarios(disrupt_type, disrupt_steps, scen_path)
    resiliency_disruptions.disrupt_network(disrupt_type, disrupt_steps, scen_path, edges_remove)
    results = resiliency_disruptions.run_o_steps(disrupt_type, disrupt_steps, scen_path, PYTHON, FTOT)
    results

## Generate disruption result report

Run `compile_report.py`, which generates the `Disruption_Results.html` report.


In [ ]:
import compile_report

compile_report.render(scen_path)